In [18]:
import os
import json
import math
from collections import Counter

def compute_entropy(action_list):
    counts = Counter(str(a) for a in action_list)
    total = sum(counts.values())
    return -sum((c / total) * math.log(c / total, 2) for c in counts.values() if c > 0)

def detect_coordination_conflicts(observations, actions):
    conflict_count = 0
    for t in range(1, len(observations)):
        pos0 = tuple(observations[t]["players"][0]["position"])
        pos1 = tuple(observations[t]["players"][1]["position"])
        act0 = actions[t][0]
        act1 = actions[t][1]

        both_interact = (
            act0 == "INTERACT" or (isinstance(act0, list) and "INTERACT" in act0)
        ) and (
            act1 == "INTERACT" or (isinstance(act1, list) and "INTERACT" in act1)
        )

        near_same_tile = abs(pos0[0] - pos1[0]) + abs(pos0[1] - pos1[1]) <= 1

        if both_interact and near_same_tile:
            conflict_count += 1

    return conflict_count

def analyze_overcooked_json(filepath):
    with open(filepath, 'r') as f:
        data = json.load(f)

    observations = data['ep_observations'][0]
    actions = data['ep_actions'][0]
    rewards = data['ep_rewards'][0]

    total_reward = sum(rewards)
    episode_length = len(observations)
    soups_delivered = rewards.count(20)
    reward_events = [(i, r) for i, r in enumerate(rewards) if r > 0]

    interact_counts = [0, 0]
    idle_counts = [0, 0]
    action_sequences = [[], []]
    division_of_labor = [0, 0]
    soup_holder_at_reward = [[], []]
    reaction_times = []

    for t, frame_actions in enumerate(actions):
        for i in [0, 1]:
            act = frame_actions[i]
            action_sequences[i].append(act)
            if act == "INTERACT" or (isinstance(act, list) and "INTERACT" in act):
                interact_counts[i] += 1
            elif act == [0, 0]:
                idle_counts[i] += 1

    for t, r in reward_events:
        for dt in range(3):
            check_t = max(t - dt, 0)
            for agent in [0, 1]:
                held = observations[check_t]["players"][agent].get("held_object")
                if held and held.get("name") == "soup":
                    soup_holder_at_reward[agent].append(t)
                    division_of_labor[agent] += 1
                    reaction_times.append(dt)
                    break

    entropies = [compute_entropy(seq) for seq in action_sequences]
    avg_reaction_time = sum(reaction_times) / len(reaction_times) if reaction_times else None
    reward_efficiency = total_reward / episode_length if episode_length else 0
    conflict_count = detect_coordination_conflicts(observations, actions)

    print(f"\n📄 File: {os.path.basename(filepath)}")
    print(f"- Episode Length: {episode_length}")
    print(f"- Total Reward: {total_reward}")
    print(f"- Soups Delivered: {soups_delivered}")
    print(f"- Reward Efficiency: {reward_efficiency:.2f}")
    print(f"- INTERACTs: Agent 0 = {interact_counts[0]}, Agent 1 = {interact_counts[1]}")
    print(f"- Idle Steps: Agent 0 = {idle_counts[0]}, Agent 1 = {idle_counts[1]}")
    print(f"- Action Entropy: Agent 0 = {entropies[0]:.2f}, Agent 1 = {entropies[1]:.2f}")
    print(f"- Deliveries (approx): Agent 0 = {division_of_labor[0]}, Agent 1 = {division_of_labor[1]}")
    print(f"- Coordination Conflict Count: {conflict_count}")
    print(f"- Avg Reaction Time: {avg_reaction_time if avg_reaction_time else 'N/A'}")

    if reward_events:
        print(f"- Reward Events:")
        for t, r in reward_events:
            print(f"  • Step {t}: +{r}")
    else:
        print("- No rewards earned.")
# Correct folder path (change as needed)
folder = "Cramped_room/"

# Run analysis
for filename in os.listdir(folder):
    if filename.endswith(".json"):
        analyze_overcooked_json(os.path.join(folder, filename))



📄 File: Human-aware PPO agent vs Human Keyboard Input 60 sec Cramped Room.json
- Episode Length: 404
- Total Reward: 120
- Soups Delivered: 6
- Reward Efficiency: 0.30
- INTERACTs: Agent 0 = 107, Agent 1 = 34
- Idle Steps: Agent 0 = 28, Agent 1 = 293
- Action Entropy: Agent 0 = 2.49, Agent 1 = 1.47
- Deliveries (approx): Agent 0 = 6, Agent 1 = 12
- Coordination Conflict Count: 5
- Avg Reaction Time: 1.0
- Reward Events:
  • Step 40: +20
  • Step 101: +20
  • Step 153: +20
  • Step 239: +20
  • Step 315: +20
  • Step 373: +20

📄 File: Human-aware PPO agent vs Human-aware PPO agent 60 sec Cramped Room.json
- Episode Length: 404
- Total Reward: 180
- Soups Delivered: 9
- Reward Efficiency: 0.45
- INTERACTs: Agent 0 = 91, Agent 1 = 96
- Idle Steps: Agent 0 = 60, Agent 1 = 72
- Action Entropy: Agent 0 = 2.55, Agent 1 = 2.55
- Deliveries (approx): Agent 0 = 15, Agent 1 = 12
- Coordination Conflict Count: 12
- Avg Reaction Time: 1.0
- Reward Events:
  • Step 38: +20
  • Step 73: +20
  • Step